In [ ]:
# encoder output에
# 평탄화(FC layer) -> 활성화함수 활성화 -> 분류기(softmax) 함수로 분류

In [1]:
import whisper
from datasets import load_dataset
from torch import nn
import collections
import torch, gc

/opt/conda/envs/lv3/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-01 07:33:02.914118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-01 07:33:04.019348: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-01 07:33:04.019429: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dler

In [2]:
# 모델 불러오기
whisper_model = whisper.load_model('large')  # model_name : large(default)

In [3]:
# encoder만 사용하기
encoder = whisper_model.encoder.to(torch.device("cuda"))

In [4]:
# Freeze our feature parameters : encoder 가중치 바뀌지 않도록 설정
for param in encoder.parameters():
  param.requires_grad = False

In [5]:
my_classifier  = nn.Sequential(
    nn.Linear(1280, 512),   # FC 1
    nn.GELU(approximate='none'),
    nn.Linear(512, 7)        # FC 2
)

# Layer 추가하기
model = nn.Sequential(collections.OrderedDict([
    ('encoder', encoder),
    ('classifier', my_classifier)
]))

In [22]:
# 모델 안에 선언되어있는 다양한 weight들의 key값
# model.state_dict().keys()

---

In [7]:
# 학습 데이터 불러오기
dataset_1 = load_dataset("anton-l/superb_demo", "er", split="session1")
dataset_1

Found cached dataset superb_demo (/opt/ml/.cache/huggingface/datasets/anton-l___superb_demo/er/1.9.0/77d23894ff429329a7fe80f9007cabb0deec321316f8dda1a1e9d10ffa089d08)


Dataset({
    features: ['file', 'audio', 'label'],
    num_rows: 6
})

In [10]:
class re_Dataset(torch.utils.data.Dataset):
    """Dataset 구성을 위한 class."""

    def __init__(self, dataset):
        self.dataset = dataset['file']
        self.dataset = dataset['label']

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [8]:
audio_array = []
label = []
for data in dataset_1 :
    gc.collect()
    torch.cuda.empty_cache()    
    
    print(data['file'])
    audio = whisper.load_audio(data['file'])            # 음성 파일을 sr=16000으로 자른 파일
    audio_array.append((audio))
    label.append(data['label'])

/opt/ml/.cache/huggingface/datasets/downloads/extracted/1e773cbd2adb373017db224742a17d9bd8924b5f5d9502862827d3b4ccf24498/Session1/sentences/wav/Ses01F_impro03/Ses01F_impro03_F013.wav
/opt/ml/.cache/huggingface/datasets/downloads/extracted/1e773cbd2adb373017db224742a17d9bd8924b5f5d9502862827d3b4ccf24498/Session1/sentences/wav/Ses01F_impro04/Ses01F_impro04_F000.wav
/opt/ml/.cache/huggingface/datasets/downloads/extracted/1e773cbd2adb373017db224742a17d9bd8924b5f5d9502862827d3b4ccf24498/Session1/sentences/wav/Ses01F_impro05/Ses01F_impro05_F015.wav
/opt/ml/.cache/huggingface/datasets/downloads/extracted/1e773cbd2adb373017db224742a17d9bd8924b5f5d9502862827d3b4ccf24498/Session1/sentences/wav/Ses01M_impro03/Ses01M_impro03_M007.wav
/opt/ml/.cache/huggingface/datasets/downloads/extracted/1e773cbd2adb373017db224742a17d9bd8924b5f5d9502862827d3b4ccf24498/Session1/sentences/wav/Ses01M_impro04/Ses01M_impro04_M023.wav
/opt/ml/.cache/huggingface/datasets/downloads/extracted/1e773cbd2adb373017db224742a17

In [11]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from transformers import Trainer, TrainingArguments
import os
import pytorch_lightning as pl

In [12]:
ck_dir_path = f"/opt/ml/final/model/hidden_model_test"
# if not os.path.exists(ck_dir_path):
#     os.makedirs(ck_dir_path)

In [13]:
 # Checkpoint
checkpoint_callback = ModelCheckpoint(
    dirpath=ck_dir_path,
    filename="{epoch}_{val_f1:.2f}",
    monitor="val_f1",
    save_top_k=1,
    mode="max",
)

In [14]:
# Earlystopping
earlystopping = EarlyStopping(monitor="val_f1", patience=3, mode="max")

In [15]:
train_dataset = re_Dataset(audio_array[:3],label[:3])
eval_dataset = re_Dataset(audio_array[3:],label[3:])

In [16]:
trainer = pl.Trainer(
    precision=16,
    accelerator="gpu",
    devices=1,
    # logger=wandb_logger,  # W&B integration
    callbacks=[earlystopping, checkpoint_callback, RichProgressBar()],
    deterministic=True,
    # limit_train_batches=0.15,  # use only 15% of training data
    # limit_val_batches = 0.01, # use only 1% of val data
    # limit_train_batches=10    # use only 10 batches of training data
    )

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
def compute_metrics(pred):
    """validation을 위한 metrics function"""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    probs = pred.predictions

    # calculate accuracy using sklearn's function
    f1 = klue_re_micro_f1(preds, labels)
    auprc = klue_re_auprc(probs, labels)
    acc = accuracy_score(labels, preds)  # 리더보드 평가에는 포함되지 않습니다.

    return {
        "micro f1 score": f1,
        "auprc": auprc,
        "accuracy": acc,
    }

In [ ]:
model

In [17]:
training_args = TrainingArguments(
        output_dir="./results",  # output directory
        save_total_limit=5,  # number of total save model.
        save_steps=500,  # model saving step.
        num_train_epochs=20,  # total number of training epochs
        learning_rate=5e-5,  # learning_rate
        per_device_train_batch_size=16,  # batch size per device during training
        per_device_eval_batch_size=16,  # batch size for evaluation
        warmup_steps=500,  # number of warmup steps for learning rate scheduler
        weight_decay=0.01,  # strength of weight decay
        logging_dir="./logs",  # directory for storing logs
        logging_steps=100,  # log saving step.
        evaluation_strategy="steps",  # evaluation strategy to adopt during training
        # `no`: No evaluation during training.
        # `steps`: Evaluate every `eval_steps`.
        # `epoch`: Evaluate every end of epoch.
        eval_steps=500,  # evaluation step.
        load_best_model_at_end=True,
    )

In [20]:
trainer = Trainer(
        model=model,  # the instantiated 🤗 Transformers model to be trained
        args=training_args,  # training arguments, defined above
        train_dataset=train_dataset,  # training dataset
        eval_dataset=eval_dataset,  # evaluation dataset
        compute_metrics=compute_metrics,  # define metrics function
    )

In [21]:
trainer.train()

/opt/conda/envs/lv3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20
  Number of trainable parameters = 659463


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/envs/lv3/lib/python3.8/site-packages/transformers/trainer.py:1543 in train            │
│                                                                                                  │
│   1540 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1541 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1542 │   │   )                                                                                 │
│ ❱ 1543 │   │   return inner_training_loop(                                                       │
│   1544 │   │   │   args=args,                                                                    │
│   1545 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1546 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/conda/envs/lv3/lib/python3.8/site-packages/transformers/trainer.py:1765 in                  │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1762 │   │   │   │   self._load_rng_state(resume_from_checkpoint)                              │
│   1763 │   │   │                                                                                 │
│   1764 │   │   │   step = -1                                                                     │
│ ❱ 1765 │   │   │   for step, inputs in enumerate(epoch_iterator):                                │
│   1766 │   │   │   │                                                                             │
│   1767 │   │   │   │   # Skip past any already trained steps if resuming training                │
│   1768 │   │   │   │   if steps_trained_in_current_epoch > 0:                                    │
│                                                                                                  │
│ /opt/conda/envs/lv3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:628 in __next__   │
│                                                                                                  │
│    625 │   │   │   if self._sampler_iter is None:                                                │
│    626 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    627 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  628 │   │   │   data = self._next_data()                                                      │
│    629 │   │   │   self._num_yielded += 1                                                        │
│    630 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    631 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ /opt/conda/envs/lv3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:671 in _next_data │
│                                                                                                  │
│    668 │                                                   